In [ ]:
!pip install selenium
!pip install webdriver-manager

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options


chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


driver.get("https://worldathletics.org")
print(driver.title)

driver.quit()


import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from urllib3.exceptions import InsecureRequestWarning
import urllib3
urllib3.disable_warnings(InsecureRequestWarning)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
urls = [
    "https://worldathletics.org/records/toplists/road-running/marathon/outdoor/women/senior/2017?regionType=world&drop=all&fiftyPercentRule=all&bestResultsOnly=false",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/women/senior/2019?regionType=world&windReading=regular&bestResultsOnly=true",
    "https://worldathletics.org/records/toplists/road-running/marathon/outdoor/women/senior/2022?regionType=world&drop=all&fiftyPercentRule=all&bestResultsOnly=false",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/women/senior/2021",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/men/senior/2019",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/men/senior/2017",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/men/senior/2021",
    "https://worldathletics.org/records/toplists/road-running/marathon/outdoor/men/senior/2022?regionType=world&drop=all&fiftyPercentRule=all&bestResultsOnly=false",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/women/senior/2016?regionType=world&windReading=regular&bestResultsOnly=true",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/women/senior/2014",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/men/senior/2016",
    "https://worldathletics.org/records/toplists/road-running/marathon/outdoor/men/senior/2014",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/women/senior/2020",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/men/senior/2020?regionType=world&windReading=regular&bestResultsOnly=true",
    "https://worldathletics.org/records/toplists/road-running/marathon/outdoor/women/senior/2025",
    "https://worldathletics.org/records/toplists/road-running/marathon/outdoor/men/senior/2025",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/women/senior/2024",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/men/senior/2024",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/women/senior/2023",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/men/senior/2023",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/men/senior/2009",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/women/senior/2011?regionType=countries&region=usa&bestResultsOnly=true&maxResultsByCountry=all&eventId=10229534&ageCategory=senior",
    "https://worldathletics.org/records/toplists/road-running/marathon/outdoor/men/senior/2011?regionType=world&drop=all&fiftyPercentRule=all&bestResultsOnly=false",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/women/senior/2014",
    "https://worldathletics.org/records/toplists/road-running/marathon/outdoor/men/senior/2014",
    "https://worldathletics.org/records/toplists/road-running/marathon/all/women/senior/2009"
]
def extract_info_from_url(url):
    year_match = re.search(r'/(\d{4})', url)
    gender = 'Female' if '/women/' in url else 'Male'
    year = int(year_match.group(1)) if year_match else None
    covid_status = 'Before COVID' if year < 2020 else ('During COVID' if year == 2020 else 'After COVID')
    return year, gender, covid_status

all_data = []
for base_url in urls:
    year, gender, covid_status = extract_info_from_url(base_url)
    print(f"\n Scraping {gender} | {year} | {covid_status}")
    empty_count = 0
    for page in range(1, 11):  
        if "page=" in base_url:
            full_url = re.sub(r'page=\d+', f'page={page}', base_url)
        elif '?' in base_url:
            full_url = f"{base_url}&page={page}"
        else:
            full_url = f"{base_url}?page={page}"

        print(f"  ➤ Page {page}: {full_url}")
        try:
            response = requests.get(full_url, headers=headers, verify=False, timeout=10)
            print(f"    Status Code: {response.status_code}")
            soup = BeautifulSoup(response.content, "html.parser")
            rows = soup.select("table.records-table tbody tr")
            print(f"     Found {len(rows)} rows.")
            if not rows:
                empty_count += 1
                if empty_count >= 2: 
                    print("     No data on two pages in a row. Stopping.")
                    break
                else:
                    print("    Empty page. Might still be more pages.")
                    continue
            else:
                empty_count = 0

            for row in rows:
                cols = row.find_all("td")
                if len(cols) < 10:
                    continue
                pos = cols[0].text.strip()
                mark = cols[1].text.strip()
                athlete = cols[3].text.strip()
                dob = cols[4].text.strip()
                country = cols[5].text.strip()
                venue = cols[8].text.strip()
                date = cols[9].text.strip()
                try:
                    birth_year = int(dob.split('-')[0])
                    age = year - birth_year
                except:
                    age = None
                all_data.append({
                    "Mark": mark,
                    "POS": pos,
                    "Athlete": athlete,
                    "Country": country,
                    "Venue": venue,
                    "Date": date,
                    "Age": age,
                    "Sex": gender,
                    "COVID_STATUS": covid_status
                })

            time.sleep(1.5) 
        except Exception as e:
            print(f"     Error scraping page {page}: {e}")
            break

df = pd.DataFrame(all_data)
df.to_csv("mdata.csv", index=False)
print("\n Done. Data saved to 'mdata.csv'")
